Chapter 5: Training ML models with Serotiny
Suggested: Alex, Gui
- Quick explanation of serotiny’s yaml-based task formulation
- Show how one can start a simple training based on 2D images to classify, e.g. edge vs. non-edge cells
- Show how one can load and apply the trained model
- Show how to bring in a pretrained model (2D RESNET)
- Show that we can use the latent space from 3D images (which has been precomputed and stored)



# **Serotiny in a nutshell**

`serotiny` is a Python package and framework to help you create configurable and reproducible DL projects. It uses [hydra](https://hydra.cc/) for configurability, [MLFlow](https://mlflow.org/) for experiment tracking,
and [Pytorch Lightning](https://pytorchlightning.ai/) for DL model training/testing/predictions.

## **Project structure**
With `serotiny` a DL project has a predefined structure (which this repo already complies with). To start a new project with the appropriate structure, you can use the [serotiny-project cookiecutter](https://github.com/allencellmodeling/serotiny-project-cookiecutter)

A serotiny project contains a Python package, and a config folder. This config folder is composed of 5 config groups:

- `data` : where we configure access to datasets

- `model` : where we define and parameterize models/networks

- `trainer` : where we setup the Pytorch Lightning trainer

- `trainer/callbacks` : where we (optionally) add a list of callbacks to be run in train/test/prediction

- `mlflow`: where we configure access to a MLFlow server, where our experiments and results will be tracked and stored

## **`serotiny` commands**
Aside from the predefined structure and config folder, `serotiny` has set of commands which know how to read a project's configuration (and override it)
and execute DL tasks.

For example, we could train a model using the a model config called `my_classifier` (which would live in `config/model/my_classifier.yaml`), and a data config
called `my_train_data` (which would live in `config/data/my_train_data.yaml`) and overriding some of the `mlflow` config parameters.
<br><small>Note: Because we didn't specify a top-level `mlflow` config, i.e. we didn't do `mlflow=...`, `serotiny` will use the default config, which lives in `config/mlflow/default.yaml`</small>

```
$ serotiny train model=my_classifier data=my_train_data mlflow.experiment_name=some_experiment mlflow.run_name=1st_run
```

Once the model finishes training, we could use it to make predictions on a different dataset, configured in `my_predict_data`

```
$ serotiny predict model=my_classifier data=my_predict_data mlflow.experiment_name=some_experiment mlflow.run_name=1st_run
```


# **Getting started**

## **Creating a dataset**

In [ ]:
import logging
logging.getLogger("bfio").setLevel(logging.ERROR)
logging.getLogger("bfio.backends").setLevel(logging.ERROR)
logging.getLogger("aicsimageio").setLevel(logging.ERROR)

from upath import UPath as Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nbvv

from serotiny.io.image import image_loader
from cytodata_aics.io_utils import rescale_image

### Load the manifest and explore dimensions

In [ ]:
cells_df = pd.read_parquet("s3://variance-dataset/processed/manifest.parquet")
print(f"Number of cells: {len(cells_df)}")
print(f"Number of columns: {len(cells_df.columns)}")

### Make a simple dataset of edge vs. non-edge cells

In [ ]:
from serotiny.transforms.dataframe.transforms import split_dataframe

Path("/home/aicsuser/serotiny_data/").mkdir(parents=True, exist_ok=True)

n = 1000 # number of cells per class
cells_edgeVSnoedge = cells_df.groupby("edge_flag").sample(n)

# Add the train, test and validate split
cells_edgeVSnoedge = split_dataframe(dataframe=cells_edgeVSnoedge, train_frac=0.7, val_frac=0.2, return_splits=False)

cells_edgeVSnoedge.to_csv("/home/aicsuser/serotiny_data/cells_edgeVSnoedge.csv") 
print(f"Number of cells: {len(cells_edgeVSnoedge)}")
print(f"Number of columns: {len(cells_edgeVSnoedge.columns)}")

### Visualize some non-edge cell

In [ ]:
some_cell = cells_edgeVSnoedge[cells_edgeVSnoedge.edge_flag == 0].sample(1).iloc[0]

img = image_loader(
    some_cell.registered_path,
    transform=rescale_image,
    return_as_torch=False
)

img.shape

In [ ]:
plt.imshow(img[1].max(axis=0))
plt.show()

### Visualize some edge cell

In [ ]:
some_cell = cells_edgeVSnoedge[cells_edgeVSnoedge.edge_flag == 1].sample(1).iloc[0]

img = image_loader(
    some_cell.registered_path,
    transform=rescale_image,
    return_as_torch=False
)

img.shape

In [ ]:
plt.imshow(img[1].max(axis=1))
plt.show()

In [ ]:
some_cell[[col for col in some_cell.index if "fits" in col]]

## **Configuring serotiny**

**TODO** general blurp about config syntax

#### `data` config

As refered to above, `serotiny` requires you to have configured 5 modules. The first one we will look into is the `data` module.

**TODO** Incrementally build up this yaml here in the markdown

```
_target_: serotiny.datamodules.ManifestDatamodule

path: /home/aicsuser/serotiny_data/cells_edgeVSnoedge.csv

batch_size: 64
num_workers: 1
loaders:
  id:
    _target_: serotiny.io.dataframe.loaders.LoadColumn
    column: CellId
    dtype: int
  class:
    _target_: serotiny.io.dataframe.loaders.LoadColumn
    column: edge_flag
    dtype: float32
  image:
    _target_: serotiny.io.dataframe.loaders.LoadImage
    column: max_projection_z
    select_channels: ['membrane']  
    
split_column: "split"
```

#### `model` config

The next module we'll  configure is the `model` module.

**TODO** Incrementally build up this yaml here in the markdown

**TODO/Recommendation** Let's use the long-form yaml config, instead of yaml variables (and keep this version under the Advanced subsection below)


```
_aux_: 
  _a: &hidden_channels 4
  _b: &kernel_size 3
  _c: &conv_block
    _target_: torch.nn.Sequential
    _args_:
      - _target_: torch.nn.LazyConv2d
        out_channels: *hidden_channels
        kernel_size: *kernel_size
        stride: 1
      - _target_: torch.nn.LeakyReLU
      - _target_: torch.nn.LazyBatchNorm2d

_target_: serotiny.models.BasicModel
x_label: image
y_label: class
network:
  _target_: torch.nn.Sequential
  _args_:
    - *conv_block
    - *conv_block
    - *conv_block
    - _target_: serotiny.networks.layers.Flatten
    - _target_: torch.nn.LazyLinear
      out_features: 1
    - _target_: torch.nn.Sigmoid
    
loss:
  _target_: torch.nn.BCELoss
  
# a function used by `serotiny predict` to store the results of feeding data through the model
save_predictions:
  _target_: cytodata_aics.model_utils.save_predictions_classifier
  _partial_: true

# fields to include in the output for each batch
fields_to_log:
  - id
 
```

##### **Advanced version**

This version uses some YAML tricks and the `_aux_` section (which is ignored by `serotiny`) to
more flexibly build models

```
_aux_: 
  _a: &hidden_channels 4
  _b: &kernel_size 3
  _c: &conv_block
    _target_: torch.nn.Sequential
    _args_:
      - _target_: torch.nn.LazyConv2d
        out_channels: *hidden_channels
        kernel_size: *kernel_size
        stride: 1
      - _target_: torch.nn.LeakyReLU
      - _target_: torch.nn.LazyBatchNorm2d

_target_: serotiny.models.BasicModel
x_label: image
y_label: class
network:
  _target_: torch.nn.Sequential
  _args_:
    - *conv_block
    - *conv_block
    - *conv_block
    - _target_: serotiny.networks.layers.Flatten
    - _target_: torch.nn.LazyLinear
      out_features: 1
    - _target_: torch.nn.Sigmoid
    
loss:
  _target_: torch.nn.BCELoss
  
  
# a function used by `serotiny predict` to store the results of feeding data through the model
save_predictions:
  _target_: cytodata_aics.model_utils.save_predictions_classifier
  _partial_: true

# fields to include in the output for each batch
fields_to_log:
  - id
```

#### `trainer` config

####  `trainer/callbacks` config

#### `mlflow` config

## **Trainining a model**

In [ ]:
# we need the commands we type to be with respect to the project root
# (because that's what `serotiny` expects) so we change directories here,
# so we can run commands within the notebook
import os
os.chdir("/home/aicsuser/cytodata-hackathon-base")

In [ ]:
from datetime import datetime

# util to avoid referring to the same run unintentionally
now_str = lambda : datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
run_name = f"some_2d_run_{now_str()}"

!serotiny train \
    model=example_classifier_2d \
    data=example_dataloader_2d \
    mlflow=internal_mlflow \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    trainer.gpus=[0] \
    trainer.max_epochs=2

## **Using  a trained model to make predictions**

In [ ]:
!serotiny predict \
    model=example_classifier_2d \
    data=example_dataloader_2d \
    mlflow=internal_mlflow \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    trainer.gpus=[0]

### Retrieving predictions from MLFlow

In [ ]:
import mlflow
from serotiny.ml_ops.mlflow_utils import download_artifact

In [ ]:
mlflow.set_tracking_uri("https://mlflow.a100.int.allencell.org")

with download_artifact("predictions/model_predictions.csv", experiment_name="cytodata_chapter5", run_name="some_2d_run_20220913_205333") as path:
    predictions_df = pd.read_csv(path)

### **Training a 3d model**

**TODO** Make it an exercise to create the 3d versions of the configs

In [ ]:
run_name = f"some_3d_run_{now_str()}"

!serotiny train \
    model=example_classifier_3d \
    data=example_dataloader_3d \
    mlflow=internal_mlflow \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    trainer.gpus=[0] \
    trainer.max_epochs=2 